**Note**: When running this notebook on SageMaker Studio, you should make sure the 'SageMaker JumpStart Tensorflow 1.0' image/kernel is used. You can run run all cells at once or step through the notebook.

In [1]:
! pip3 install gym

from io import BytesIO
import time
import sys
sys.path.append("../BattlesnakeGym")
import json
import boto3
import botocore
import PIL.Image
import sagemaker

import numpy
import gym
from gym import wrappers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
from importlib import reload
from IPython import display
import ipywidgets as widgets
from IPython.display import display as i_display

from heuristics_utils import simulate
from battlesnake_gym.snake_gym import BattlesnakeGym

     |████████████████████████████████| 1.5 MB 8.7 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616829 sha256=4e92c24ac29da81fd5426725daacaf8a5c18d2ce63e552f1fb662981cf9aa852
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f8/c7/3c/7ad569d779e750220d17a44f731411f0ad79b7b123a2e3a02e
Successfully built gym



# Introduction
In this notebook, you can take the model that you've developed (or the pre-trained model provided) and develop heuristics to edit your snake's behaviour.

### Define the openAI gym
Optionally, you can define the initial game state (the situation simulator) of the snakes and food.
To use the initial state, set `USE_INITIAL_STATE = True` and enter the desired coordinates of the snake and food using the initial_state dictionary. The dictionary follows the same format as the battlesnake API.

In [2]:
USE_INITIAL_STATE = False

# Sample initial state for the situation simulator
initial_state = {
    "turn": 4,
    "board": {
        "height": 11,
        "width": 11,
        "food": [
            {
                "x": 1,
                "y": 3
            }
        ],
        "snakes": [{
                        "health": 90,
                        "body": [{"x": 3, "y": 0}],
                    }, 
                    {
                        "health": 90,
                        "body": [{"x": 6, "y": 0}],
                    },
                    {
                        "health": 90,
                        "body": [{"x": 2, "y": 5}],
                    },
                    {
                        "health": 90,
                        "body": [{"x": 6, "y": 4}],
                    },
                    {
                        "health": 90,
                        "body": [{"x": 7, "y": 3}],
                    },
                  ]

    }
}

if USE_INITIAL_STATE == False:
    initial_state = None

The parameters here must match the ones provided during training (except initial_state)

In [3]:
map_size = (11, 11)
number_of_snakes = 5
random_snake = True  # If this is set to true, the simulator will not use the network but just output random actions
env = BattlesnakeGym(map_size=map_size, number_of_snakes=number_of_snakes, observation_type="max-bordered-51s", 
                     initial_game_state=initial_state)

# Load the trained model
Load the RLlib models.

In [4]:
%cd inference
!mkdir output
!tar -C output -xvf model.tar.gz
%cd ..

/home/ec2-user/SageMaker/RLlibEnv/inference
params.json
checkpoint.tune_metadata
1/
1/saved_model.pb
1/variables/
1/variables/variables.index
1/variables/variables.data-00000-of-00002
1/variables/variables.data-00001-of-00002
checkpoint
/home/ec2-user/SageMaker/RLlibEnv


In [5]:
model_filepath = "inference/output/1/"
if random_snake:
    net = None
else:
    imported = tf.saved_model.load(model_filepath)
    net = imported.signatures["serving_default"]

In [6]:
# Clean up the model
!rm -r inference/output

# Simulation loop

Run a simulation with the environment with the heuritics that you wrote. 
To edit the heuristics, edit the file `RLlibEnv/inference/src/battlesnake_heuristics`.
Note that you can track the progress of your work with git.

In [50]:
import inference.inference_src.battlesnake_heuristics
reload(inference.inference_src.battlesnake_heuristics)
from inference.inference_src.battlesnake_heuristics import MyBattlesnakeHeuristics

heuristics = MyBattlesnakeHeuristics()
infos, rgb_arrays, actions, heuristics_remarks, json_array = simulate(env, net, heuristics, number_of_snakes, random_snake)

snake 0
turn 2
[[0.97161258 0.17578465 0.75570695 0.56461495]]
==wall==
[True, True, True, True]
==forbidden==
[True, True, True, True]
==food==
[True, True, True, True]
[[0.97161258 0.17578465 0.75570695 0.56461495]]
0
====
snake 1
turn 2
[[0.43417687 0.61503671 0.2126324  0.20902611]]
==wall==
[True, True, True, True]
==forbidden==
[True, True, True, True]
==food==
[True, True, True, True]
[[0.43417687 0.61503671 0.2126324  0.20902611]]
1
====
snake 2
turn 2
[[0.51547415 0.67042004 0.1547638  0.46652116]]
==wall==
[True, True, True, True]
==forbidden==
[True, True, True, True]
==food==
[True, True, True, True]
[[0.51547415 0.67042004 0.1547638  0.46652116]]
1
====
snake 3
turn 2
[[0.74854783 0.25962044 0.76280241 0.92008007]]
==wall==
[True, True, True, True]
==forbidden==
[True, True, True, True]
==food==
[True, True, True, True]
[[0.74854783 0.25962044 0.76280241 0.92008007]]
3
====
snake 4
turn 2
[[0.39598205 0.60457159 0.21491338 0.4006084 ]]
==wall==
[True, True, True, True]
==f

[True, True, False, True]
==forbidden==
[False, True, True, True]
==food==
[True, True, True, True]
[[0.         0.95104791 0.         0.90078773]]
1
====
snake 1
turn 17
[[0.64725781 0.33314677 0.96141589 0.93983322]]
==wall==
[True, True, True, True]
==forbidden==
[True, True, False, True]
==food==
[True, True, True, True]
[[0.64725781 0.33314677 0.         0.93983322]]
3
====
snake 3
turn 17
[[0.29643325 0.10964669 0.39312529 0.68393833]]
==wall==
[True, True, True, False]
==forbidden==
[True, False, True, True]
==food==
[True, True, True, True]
[[0.29643325 0.         0.39312529 0.        ]]
2
====
snake 4
turn 17
[[0.90375147 0.39529246 0.30508533 0.39019651]]
==wall==
[True, True, False, True]
==forbidden==
[True, True, True, False]
==food==
[True, True, True, True]
[[0.90375147 0.39529246 0.         0.        ]]
0
====
snake 0
turn 18
[[0.19089794 0.77458182 0.86901305 0.54309583]]
==wall==
[True, True, False, True]
==forbidden==
[False, True, True, True]
==food==
[True, True, T

snake 3
turn 39
[[0.48619982 0.69296687 0.49622106 0.21369322]]
==wall==
[True, True, True, True]
==forbidden==
[True, False, True, True]
==food==
[True, True, True, True]
[[0.48619982 0.         0.49622106 0.21369322]]
2
====
snake 4
turn 39
[[0.71656333 0.39061144 0.32851902 0.75106745]]
==wall==
[True, True, True, True]
==forbidden==
[True, False, True, True]
==food==
[True, True, True, True]
[[0.71656333 0.         0.32851902 0.75106745]]
3
====
snake 3
turn 40
[[0.16309404 0.16681631 0.49190584 0.22859367]]
==wall==
[True, True, True, True]
==forbidden==
[True, True, True, False]
==food==
[True, True, True, True]
[[0.16309404 0.16681631 0.49190584 0.        ]]
2
====
snake 4
turn 40
[[0.6783302  0.66592668 0.95413459 0.79318797]]
==wall==
[True, True, True, True]
==forbidden==
[True, True, False, True]
==food==
[True, True, True, True]
[[0.6783302  0.66592668 0.         0.79318797]]
3
====
snake 3
turn 41
[[0.2168607  0.05444991 0.93627051 0.93252139]]
==wall==
[True, True, True, 

# Playback the simulation

Defines the user interface of the simulator.

In [51]:
def get_env_json():
    if slider.value < len(json_array):
        return json_array[slider.value]
    else:
        return ""
    
def play_simulation(_):
    for i in range(slider.value, len(rgb_arrays) - slider.value - 1):
        slider.value = slider.value + 1
        display_image(slider.value)
        time.sleep(0.2)

def on_left_button_pressed(_):
    if slider.value > 0:
        slider.value = slider.value - 1 
    display_image(slider.value)

def on_right_button_pressed(_):
    if slider.value < len(rgb_arrays):
        slider.value = slider.value + 1 
    display_image(slider.value)
        
def display_image(index):  
    if index >= len(rgb_arrays):
        return
    info = infos[index]
    action = actions[index]
    heuristics = heuristics_remarks[index]
    snake_colours = env.snakes.get_snake_colours()
        
    line_0 = [widgets.Label("Turn count".format(info["current_turn"])),
                 widgets.Label("Snake")]
    
    line_1 = [widgets.Label(""), widgets.Label("Health")]
    
    line_2 = [widgets.Label("{}".format(info["current_turn"])), 
              widgets.Label("Action")]
    
    line_3 = [widgets.Label(""), widgets.Label("Gym remarks")]
    
    line_4 = [widgets.Label(""), widgets.Label("Heur. remarks")]

    action_convertion_dict = {0: "Up", 1: "Down", 2: "Left", 3: "Right", 4: "None"}
    for snake_id in range(number_of_snakes):
        snake_health = "{}".format(info["snake_health"][snake_id])
        snake_health_widget = widgets.Label(snake_health)
        snake_action = "{}".format(action_convertion_dict[action[snake_id]])
        snake_action_widget = widgets.Label(snake_action)

        snake_colour = snake_colours[snake_id]
        hex_colour = '#%02x%02x%02x' % (snake_colour[0], snake_colour[1], snake_colour[2])
        snake_colour_widget = widgets.HTML(value = f"<b><font color="+hex_colour+">⬤</b>")

        gym_remarks = ""
        if snake_id in info["snake_info"]:
            if info["snake_info"][snake_id] != "Did not colide": 
                gym_remarks = "{}".format(info["snake_info"][snake_id])
        gym_remarks_widget = widgets.Label(gym_remarks)
        
        heuris_remarks = "{}".format(heuristics[snake_id])
        heuris_remarks_widget = widgets.Label(heuris_remarks)

        line_0.append(snake_colour_widget)
        line_1.append(snake_health_widget)
        line_2.append(snake_action_widget)
        line_3.append(gym_remarks_widget)
        line_4.append(heuris_remarks_widget)

    line_0_widget = widgets.VBox(line_0)
    line_1_widget= widgets.VBox(line_1)
    line_2_widget = widgets.VBox(line_2)
    line_3_widget = widgets.VBox(line_3)
    line_4_widget = widgets.VBox(line_4)
   
    info_widget = widgets.HBox([line_0_widget, line_1_widget, line_2_widget, line_3_widget, line_4_widget])
        
    image = PIL.Image.fromarray(rgb_arrays[index])
    f = BytesIO()
    image.save(f, "png")
    
    states_widget = widgets.Image(value=f.getvalue(), width=500)
    main_widgets_list = [states_widget, info_widget]
    
    main_widget = widgets.HBox(main_widgets_list)
    
    display.clear_output(wait=True)
    i_display(navigator)
    i_display(main_widget)
    
left_button = widgets.Button(description='◄')
left_button.on_click(on_left_button_pressed)
right_button = widgets.Button(description='►')
right_button.on_click(on_right_button_pressed)
slider = widgets.IntSlider(max=len(rgb_arrays) - 1)
play_button = widgets.Button(description='Play')
play_button.on_click(play_simulation)

navigator = widgets.HBox([left_button, right_button, slider, play_button])
display_image(index=0)

To get a JSON representation of the gym (environment), run the following function. You can also use output of the following function as an initial_state of the gym.

*Please provide this json array if you are reporting bugs in the gym*

In [9]:
get_env_json()

{'turn': 0,
 'board': {'height': 11,
  'width': 11,
  'food': [{'x': 9, 'y': 10}],
  'snakes': [{'health': 100,
    'body': [{'x': 0, 'y': 3}],
    'id': 0,
    'name': 'Snake 0'},
   {'health': 100, 'body': [{'x': 8, 'y': 5}], 'id': 1, 'name': 'Snake 1'},
   {'health': 100, 'body': [{'x': 9, 'y': 4}], 'id': 2, 'name': 'Snake 2'},
   {'health': 100, 'body': [{'x': 2, 'y': 9}], 'id': 3, 'name': 'Snake 3'},
   {'health': 100, 'body': [{'x': 1, 'y': 3}], 'id': 4, 'name': 'Snake 4'}]}}

# Deploy the SageMaker endpoint
This section will deploy your new heuristics into the SageMaker endpoint

In [10]:
with open("../stack_outputs.json") as f:
    info = json.load(f)

In [11]:
sage_session = sagemaker.session.Session()
s3_bucket = info["S3Bucket"]
role = info["SageMakerIamRoleArn"]
endpoint_instance_type = info["SagemakerInferenceInstanceType"]
print("Your sagemaker s3_bucket is s3://{}".format(s3_bucket))

Your sagemaker s3_bucket is s3://sagemaker-soln-bs-rbcsnake-bucket


In [12]:
model_data = "s3://{}/pretrainedmodels/model.tar.gz".format(s3_bucket)

# Check if model_data exists
s3_client = boto3.client('s3')

try:
    s3_client.get_object(Bucket=s3_bucket, Key="pretrainedmodels/model.tar.gz")
except botocore.errorfactory.ClientError:
    model_data = info["EndPointS3Location"]

## Deploy your new heuristics
Using the new heuristics you developed, a new SageMaker endpoint will be created.

Firstly, delete the old endpoint, model and endpoint config.

In [13]:
sm_client = boto3.client(service_name='sagemaker')
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=info['SagemakerEndPointName'])
try:
    sm_client.delete_endpoint(EndpointName=info['SagemakerEndPointName'])
    sm_client.delete_endpoint_config(EndpointConfigName=info['SagemakerEndPointName'])
    sm_client.delete_model(ModelName=info['SagemakerEndPointName'])
    ep_waiter = sm_client.get_waiter('endpoint_deleted')
    ep_waiter.wait(EndpointName=info['SagemakerEndPointName'])
except botocore.exceptions.ClientError:
    pass

# Run the following cells to create a new model and endpoint with the new heuristics

from sagemaker.tensorflow.serving import Model

model = Model(model_data=model_data,
              role=role,
              entry_point="inference.py",
              source_dir='inference/inference_src',
              framework_version='2.1.0',
              name=info['SagemakerEndPointName'],
              code_location='s3://{}//code'.format(s3_bucket)
             )

# Deploy an inference endpoint
predictor = model.deploy(initial_instance_count=1, instance_type=endpoint_instance_type, endpoint_name=info['SagemakerEndPointName'])

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------!

## Testing the new endpoint
You should see `Action to take is X`

In [14]:
import numpy as np
from time import time

state = np.zeros(shape=(1, 21, 21, 6), dtype=np.float32).tolist()

health_dict = {0: 50, 1: 50}
json = {"turn": 4,
        "board": {
                "height": 11,
                "width": 11,
                "food": [],
                "snakes": []
                },
            "you": {
                "id": "snake-id-string",
                "name": "Sneky Snek",
                "health": 90,
                "body": [{"x": 1, "y": 3}]
                }
            }

before = time()
action_mask = np.array([1, 1, 1, 1]).tolist()

action = predictor.predict({"state": state, "action_mask": action_mask,
                            "prev_action": -1, 
                           "prev_reward": -1, "seq_lens": -1,  
                           "all_health": health_dict, "json": json})
elapsed = time() - before

action_to_take = action["outputs"]["heuristisc_action"]
print("Action to take {}".format(action_to_take))
print("Inference took %.2f ms" % (elapsed*1000))

Action to take 2
Inference took 793.68 ms


# Navigation
- To go back to the introduction click [here](./1_Introduction.ipynb)
- To train a new model click [here](./2_PolicyTraining.ipynb)